Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [ ]:
# Joining of dataframes has been done post competing EDA for each of the dataframes. Hence, the sections have been shuffled


city_bike_df = pd.read_csv(r'C:\Users\aksha\Documents\LHL\LHL Assignments\Week 5\Python Statistical Modeling Project\City_Bike_Vancouver_Data_17th.csv')
yelp_df = pd.read_csv(r'C:\Users\aksha\Documents\LHL\LHL Assignments\Week 5\Python Statistical Modeling Project\Yelp_Data_17th.csv')
fs_df = pd.read_csv(r'C:\Users\aksha\Documents\LHL\LHL Assignments\Week 5\Python Statistical Modeling Project\Foursquare_Data_17th.csv')

In [ ]:
# Data Audit and cleaning for City Bike dataframe
city_bike_df.head()
city_bike_df.shape # 248,13
city_bike_df.info() # No need to change data types
city_bike_df.describe(include = 'all')


# Column wise EDA and data cleaning
# ID can be dropped, since it's repetitive and was created for joining purposes
# City and Country will be same for all the records hence it can be left as it is
# Station Id - just a unique identifier for the station, can be left as it is
# Station name - 
    # Scope for text cleaning
    # Check distribution of values and see if there are some repetitive names
    # Create geographical map of stations using the latitude and longitude for showing a density plot 
# Time stamp - It will be same for all, extract and retain the date in a cleaner format as per Vancouver timezone
# For the remaining columns check the distribution and if anomalies are present (can be correlated with the time and day of api call for example all bikes shall be available at odd hours and so)

# Other plots to be created post joining and additional columns to be created
# Correlation heatmap to establish hypothesis
# Popularity index of a station - with reference to review count, ratings and number of POIs
# Correlation between popularity index and cycle renting

In [ ]:
city_bike_df = city_bike_df.drop('id', axis=1)

In [ ]:
# Check Names for DQ issues
city_bike_df.Station_Name.value_counts() 
# all have a count of one which means no 2 stations ids have the same name 

sum(city_bike_df['Station_Name'].isnull() | city_bike_df['Station_Name'] == '')
# No junk values found in the distribution like Null, NAN, Blanks, 0 etc

# Text cleaning 
import re

def process_location_string(location_string):
    # Use regex to find the brackets and their content
    match = re.search(r'^(.*?)\(([^)]*)\)(.*)$', location_string)

    if match:
        # Extract parts of the string before, between, and after brackets
        before_brackets = match.group(1)
        brackets_content = match.group(2)
        after_brackets = match.group(3)

        # Add "near" before the brackets and remove brackets
        new_location_string = f"{before_brackets}near {brackets_content}{after_brackets}"

        return new_location_string

    # Return the original string if no match is found
    return location_string

city_bike_df['Station_Name'] = city_bike_df['Station_Name'].apply(process_location_string)

In [ ]:
# Visualize the bike stations in a geo map

color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(city_bike_df, 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Station_Name", 
                        hover_data=["Station_Name", "Total_Available_Free_Bikes"],
                        color="Total_Available_Free_Bikes",
                        color_continuous_scale=color_scale,
                        size="Total_Available_Free_Bikes",
                        zoom=8, 
                        height=600,
                        width=800)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
# Majority of the stations are in the downtown area

In [ ]:
# Extract Date and Time

city_bike_df['Timestamp'].value_counts() # Millisconds were not in consistent format for 1 record in the timestamp object

import pytz

# Function to convert UTC to Vancouver time
def convert_to_vancouver(timestamp):
    utc_datetime = pd.to_datetime(timestamp, errors='coerce')
    if not pd.isnull(utc_datetime):
        vancouver_timezone = pytz.timezone("America/Vancouver")
        vancouver_datetime = utc_datetime.replace(tzinfo=pytz.utc).astimezone(vancouver_timezone)
        vancouver_timestamp = vancouver_datetime.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3] + "Z"
        return vancouver_timestamp
    return None

# Apply the conversion function to the 'Timestamp' column
city_bike_df['Date_Time'] = city_bike_df['Timestamp'].apply(convert_to_vancouver)

city_bike_df['Vancouver_Timestamp'] = pd.to_datetime(city_bike_df['Vancouver_Timestamp'])

# Extract date and time components
city_bike_df['Date'] = city_bike_df['Vancouver_Timestamp'].dt.date
city_bike_df['Time'] = city_bike_df['Vancouver_Timestamp'].dt.time

city_bike_df.drop(['Timestamp', 'Vancouver_Timestamp'], axis = 1, inplace = True)
city_bike_df.head()

In [ ]:
# Lastly we will check for anomalies in the bike and slots data using boxplots and study their distributions

# Create a figure and a grid of subplots
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 8))

# Flatten the axes array for easier indexing
axes = axes.flatten()

# Example boxplots for different columns
sns.boxplot(y='Total_Available_Slots', data=city_bike_df, ax=axes[0])
axes[0].set_title('Total_Available_Slots')

sns.boxplot(y='Total_Available_Free_Bikes', data=city_bike_df, ax=axes[1])
axes[1].set_title('Total_Available_Free_Bikes')

sns.boxplot(y='Total_Available_EBikes', data=city_bike_df, ax=axes[2])
axes[2].set_title('Total_Available_EBikes')

sns.boxplot(y='Total_Available_Normal_Bikes', data=city_bike_df, ax=axes[3])
axes[3].set_title('Total_Available_Normal_Bikes')

sns.boxplot(y='Total_Available_Empty_Slots', data=city_bike_df, ax=axes[4])
axes[4].set_title('Total_Available_Empty_Slots')

# Adjust layout
plt.tight_layout()

# Show the plots
plt.show()

# Following columns have anomalies - Total_Available_Slots, Total_Available_Free_Bikes, Total_Available_EBikes, Total_Available_Normal_Bikes
# Skewness - All the boxplots are right skewed
# If there had been any missing values then we could not have used mean to impute the same

In [ ]:

# Create a figure and a grid of subplots
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 8))

# Flatten the axes array for easier indexing
axes = axes.flatten()

# Example histograms with a line for different columns
columns = ['Total_Available_Slots','Total_Available_Free_Bikes','Total_Available_EBikes',
'Total_Available_Normal_Bikes','Total_Available_Empty_Slots']

for i, column in enumerate(columns):
    sns.histplot(city_bike_df[column].dropna(), kde=True, ax=axes[i], color='skyblue', bins=20)
    axes[i].set_title(f'Histogram for {column}')

# Adjust layout
plt.tight_layout()

# Show the plots
plt.show()

In [ ]:
# Top 5 stations with the most number of bikes in use
city_bike_df.head()
city_bike_df['Total_Bikes_In_Use'] = city_bike_df['Total_Available_Slots'] - city_bike_df['Total_Available_Free_Bikes']
city_bike_df.iloc[city_bike_df['Total_Bikes_In_Use'].value_counts().head(5).index]['Station_Name']

Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [ ]:
# Join the dataframes

print(city_bike_df.shape)
print(yelp_df.shape)
print(fs_df.shape)

# Check column names and data types

print(city_bike_df.info())
print(yelp_df.info())
print(fs_df.info()) # Lat and Long are in float

# Check for overalpping records

print(sum(yelp_df.Latitude.isin(city_bike_df.Latitude)))
print(sum(yelp_df.Longitude.isin(city_bike_df.Longitude)))

print(sum(fs_df.Latitude.isin(city_bike_df.Latitude)))
print(sum(fs_df.Longitude.isin(city_bike_df.Longitude)))

complete_df = pd.merge(city_bike_df, yelp_df, on = ['Latitude', 'Longitude'], how = 'left')
print(complete_df.shape)
complete_df = pd.merge(complete_df, fs_df, on = ['Latitude', 'Longitude'], how = 'left')
print(complete_df.shape)

In [ ]:
# EDA Part Continued post joining the dataframes

# Data cleaning and EDA steps
# Drop id_dummy column
# Rename Yelp columns and FS columns for easier bifurcation
# Check distribution and anomalies in newly added columns
# Impute NA values
# Create new columns - Total Number of FSQ POIs, Avg number of POI in a place, Popularity index of a place (3 buckets as per the count of POIs)
# Geo map of POI per staion
# Check for correlation between following parameters -
    # Total Slots and Total POIs
    # Total Bikes in Use and Total POIs
    # Total Bikes in Use and Total number of reviews
    # Total Bikes in Use and Avg Ratings

complete_df.drop('id_dummy', axis = 1, inplace = True)

column_mapping = {'Total_POI': 'Yelp_Total_POI', 'Closed_POI': 'Yelp_Closed_POI', 'AvgRating': 'Yelp_Avg_Rating',
                 'AvgReviewCount': 'Yelp_Avg_Review_Count', 'PriceCategory': 'Yelp_Price_Category', 'AvgRating': 'Yelp_Avg_Rating',
                 'LikelyOpen': 'FSQ_Likely_Open', 'VeryLikelyOpen': 'FSQ_Very_Likely_Open', 'Unsure': 'FSQ_Unsure'}

# Rename columns using the 'rename' method
complete_df.rename(columns=column_mapping, inplace=True)



In [ ]:
# Data cleaning and EDA steps
# Drop id_dummy column
# Rename Yelp columns and FS columns for easier bifurcation
# Check distribution and anomalies in newly added columns
# Impute NA values
# Create new columns - Total Number of FSQ POIs, Avg number of POI in a place, Popularity index of a place (3 buckets as per the count of POIs)
# Geo map of POI per staion
# Check for correlation between following parameters -
    # Total Slots and Total POIs
    # Total Bikes in Use and Total POIs
    # Total Bikes in Use and Total number of reviews
    # Total Bikes in Use and Avg Ratings

complete_df.drop('id_dummy', axis = 1, inplace = True)

column_mapping = {'Total_POI': 'Yelp_Total_POI', 'Closed_POI': 'Yelp_Closed_POI', 'AvgRating': 'Yelp_Avg_Rating',
                 'AvgReviewCount': 'Yelp_Avg_Review_Count', 'PriceCategory': 'Yelp_Price_Category', 'AvgRating': 'Yelp_Avg_Rating',
                 'LikelyOpen': 'FSQ_Likely_Open', 'VeryLikelyOpen': 'FSQ_Very_Likely_Open', 'Unsure': 'FSQ_Unsure'}

# Rename columns using the 'rename' method
complete_df.rename(columns=column_mapping, inplace=True)



In [ ]:
# Distribution of Yelp Data Points

# Create a figure and a grid of subplots
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

# Flatten the axes array for easier indexing
axes = axes.flatten()

# Example histograms with a line for different columns
columns = ['Yelp_Total_POI','Yelp_Closed_POI','Yelp_Avg_Rating',
'Yelp_Avg_Review_Count']

for i, column in enumerate(columns):
    sns.histplot(complete_df[column].dropna(), kde=True, ax=axes[i], color='skyblue', bins=20)
    axes[i].set_title(f'Histogram for {column}')

# Adjust layout
plt.tight_layout()

# Show the plots
plt.show()

# Total POI is left skewed
# Closed POI has only 0s
# Avg Rating is slightly left skewed
# Avg Review Count is right skewed

# Since the Closed POI has only 0s we will be dropping it. 

complete_df.drop('Yelp_Closed_POI', axis = 1, inplace = True)

In [ ]:
# Visualize the frequency count in price category column

# Get the count of each category
price_category_counts = complete_df['Yelp_Price_Category'].value_counts()

# Create a pie chart with percentage annotations
plt.pie(price_category_counts, labels=price_category_counts.index, 
        autopct='%1.1f%%', startangle=90, colors=sns.color_palette('pastel'))

# Equal aspect ratio ensures that pie is drawn as a circle.
plt.axis('equal')

# Display the plot
plt.show()

In [ ]:
# NA Imputation in Yelp Data
# There is one record in the dataframe where the records were not found in api response. 
# After viewing the distribution, owing to skewness in data we will be imputing with median values
# In categorical column we will be imputing with the mode of the column


# Calculate median for 'Yelp_Total_POI','Yelp_Avg_Rating','Yelp_Avg_Review_Count'
median_values = complete_df[['Yelp_Total_POI','Yelp_Avg_Rating','Yelp_Avg_Review_Count']].median()

# Calculate mode for col4
mode_value = complete_df['Yelp_Price_Category'].mode().iloc[0]

# # Impute missing values using fillna
complete_df['Yelp_Total_POI'].fillna(median_values['Yelp_Total_POI'], inplace=True)
complete_df['Yelp_Avg_Rating'].fillna(median_values['Yelp_Avg_Rating'], inplace=True)
complete_df['Yelp_Avg_Review_Count'].fillna(median_values['Yelp_Avg_Review_Count'], inplace=True)
complete_df['Yelp_Price_Category'].fillna(mode_value, inplace=True)

In [ ]:
# Imputation of NANs in the FSQ fields

complete_df['FSQ_Likely_Open'].fillna(0, inplace=True)
complete_df['FSQ_Very_Likely_Open'].fillna(0, inplace=True)
complete_df['FSQ_Unsure'].fillna(0, inplace=True)

# Check for NANs
complete_df.isna().sum(axis = 0)

In [ ]:
# Create New column of Total FSQ POIs
complete_df['FSQ_Total_POI'] = complete_df['FSQ_Likely_Open'] + complete_df['FSQ_Very_Likely_Open'] + complete_df['FSQ_Unsure']

In [ ]:
# Frequency Distribution of FSQ POI
complete_df.head(25)
complete_df['FSQ_Total_POI'].value_counts() # We can see that FSQ has not given more than 10 records per lat long whereas in Yelp
# API response the range was much wider 

sns.histplot(complete_df['FSQ_Total_POI'], bins=20, kde=True, color='skyblue')
plt.title('Histogram of FSQ_Total_POI')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.show()

# the distribution is highly right-skewed, majority of values are concentrated around 10

In [ ]:
# Check for variance in POI data per Lat Long between FSQ and Yelp data
complete_df['Diff_POI_Count'] = complete_df['Yelp_Total_POI'] - complete_df['FSQ_Total_POI']
sns.histplot(complete_df['Diff_POI_Count'], bins=20, kde=True, color='skyblue')
plt.title('Histogram of Diff_POI_Count')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.show()

complete_df[['Yelp_Total_POI', 'FSQ_Total_POI']].describe()



In [ ]:
# Yelp data seems to be more complete and thus reliable compared to the FSQ data. 
# Therefore we will be considering a weighted avg of both the data sources - 90% weightage to Yelp and 10% to FSQ

complete_df['Avg_POI_Count'] = 0.9 * complete_df['Yelp_Total_POI'] + 0.1 * complete_df['FSQ_Total_POI']
# Since the sum of weights is 1 we won't be dividing with it.

# Popularity Index

# Specify the number of bins (in this case, 3 for low, medium, and high)
num_bins = 3

# Use qcut to bucket the numerical column into three categories
complete_df['Popularity_Index'] = pd.qcut(complete_df['Avg_POI_Count'], q=num_bins, labels=['Low', 'Medium', 'High'])


# Visualize the bike stations in a geo map

color_scale = {'Low': 'orange', 'Medium': 'yellow', 'High': 'red'}

# Create a numerical representation of 'Popularity_Index'
complete_df['Popularity_Index_Num'] = complete_df['Popularity_Index'].map({'Low': 1, 'Medium': 2, 'High': 3})

fig = px.scatter_mapbox(complete_df, 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Station_Name", 
                        hover_data=["Station_Name", "Avg_POI_Count"],
                        color="Popularity_Index_Num",
                        color_discrete_map=color_scale,
                        size="Popularity_Index_Num",
                        zoom=8, 
                        height=800,
                        width=800,
                        labels={'Popularity_Index_Num': 'Popularity Index'})  # Title for the legend

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
# The further we go from downtown the lesser is the popularity index of the station


In [ ]:

# Check for correlation between following parameters -
    # Total Slots and Total POIs
    # Total Bikes in Use and Total POIs
    # Total Bikes in Use and Total number of reviews
    # Total Bikes in Use and Avg Ratings
    
complete_df.columns
# Calculate the correlation matrix
corr_matrix = complete_df[['Total_Available_Slots', 'Yelp_Avg_Rating', 'Yelp_Avg_Review_Count', 'Avg_POI_Count', 'Total_Bikes_In_Use']].corr()

# Create a heatmap 
plt.figure(figsize=(6, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Heatmap')
plt.show()

# Review count and ratings are highly correlated (0.46)
# Avg POI count and review count are highly correlated (0.58)
# Since one is a subset of another, hence it's obvious that total Bikes in Use and Available slots are mildly correlated (0.33)

# DQ Check 
# Check whether the Empty slots = Total Slots - Total Free Bikes
sum(complete_df['Total_Available_Empty_Slots'] == complete_df['Total_Available_Slots'] - complete_df['Total_Available_Free_Bikes'])
# In 194 records the total available empty slots matches the difference between available slots and available free bikes



In [ ]:
print(complete_df.columns)
# Box plot of total bikes in use vs price category

plt.figure(figsize=(8, 6))
sns.boxplot(x='Yelp_Price_Category', y='Total_Bikes_In_Use', data=complete_df)
plt.title('Boxplot of Total_Bikes_In_Use by Yelp_Price_Category')
plt.show()

# We can see that the total bikes in use are in higher number within an economical POI compared to an Expensive one
# We will validate the hypothesis using a t test

# T Test (Welch's t test as both the groups have unequal sample sizes) 

# Separate data into two groups
economical_category = complete_df['Total_Bikes_In_Use'][complete_df['Yelp_Price_Category'] == 'Economical']
expensive_category = complete_df['Total_Bikes_In_Use'][complete_df['Yelp_Price_Category'] == 'Expensive']

# Perform Welch's t-test
t_statistic, p_value = ttest_ind(economical_category, expensive_category, equal_var=False)

# Print the t-test result
print(f"T-Statistic: {t_statistic}")
print(f"P-Value: {p_value}")

# Check the p-value to determine statistical significance
if p_value < 0.05:
    print("The means are significantly different.")
else:
    print("No significant difference in means.")

# We cannot reject the null hypothesis


# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [ ]:
# Create a SQLite database 
db_path = 'bike_station.db'
conn = sqlite3.connect(db_path)

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Create a table
create_table_query = '''
    CREATE TABLE IF NOT EXISTS bike_station_data (
        City TEXT NOT NULL,
        Country TEXT NOT NULL,
        Station_Id TEXT NOT NULL,
        Station_Name TEXT NOT NULL,
        Latitude REAL NOT NULL,
        Longitude REAL NOT NULL,
        Total_Available_Slots INTEGER NOT NULL,
        Total_Available_Free_Bikes INTEGER NOT NULL,
        Total_Available_EBikes INTEGER NOT NULL,
        Total_Available_Normal_Bikes INTEGER NOT NULL,
        Total_Available_Empty_Slots INTEGER NOT NULL,
        Date_Time TIMESTAMP NOT NULL,
        Total_Bikes_In_Use INTEGER NOT NULL,
        Yelp_Total_POI REAL NOT NULL,
        Yelp_Avg_Rating REAL NOT NULL,
        Yelp_Avg_Review_Count REAL NOT NULL,
        Yelp_Price_Category TEXT NOT NULL,
        FSQ_Likely_Open REAL NOT NULL,
        FSQ_Very_Likely_Open REAL NOT NULL,
        FSQ_Unsure REAL NOT NULL,
        FSQ_Total_POI REAL NOT NULL,
        Diff_POI_Count REAL NOT NULL,
        Avg_POI_Count REAL NOT NULL,
        Popularity_Index TEXT NOT NULL,
        Popularity_Index_Num TEXT NOT NULL
    );
'''
cursor.execute(create_table_query)

# Commit the changes
conn.commit()

# Push the DataFrame into the table
complete_df.to_sql('bike_station_data', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

# Create an SQLite database (this will create the database file if it doesn't exist)
db_path = 'bike_station.db'
conn = sqlite3.connect(db_path)

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Perform a SELECT * query
select_query = 'SELECT * FROM bike_station_data limit 5'
result_df = pd.read_sql(select_query, conn)

# Print the result DataFrame
print(result_df)

Look at the data before and after the join to validate your data.

In [ ]:

print(complete_df.shape[0] == city_bike_df.shape[0])
# Yes it's matching